# COCO data preprocessing

This code will download the caption anotations for coco and preprocess them into an hdf5 file and a json file. 

These will then be read by the COCO data loader in Lua and trained on.

In [1]:
# lets download the annotations from http://mscoco.org/dataset/#download
import os
os.system('wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip') # ~19MB

0

In [2]:
os.system('unzip captions_train-val2014.zip')

0

In [4]:
import json
val = json.load(open('annotations/captions_val2014.json', 'r'))
train = json.load(open('annotations/captions_train2014.json', 'r'))

In [5]:
print val.keys()
print val['info']
print len(val['images'])
print len(val['annotations'])
print val['images'][0]
print val['annotations'][0]

[u'info', u'images', u'licenses', u'annotations']
{u'description': u'This is stable 1.0 version of the 2014 MS COCO dataset.', u'url': u'http://mscoco.org', u'version': u'1.0', u'year': 2014, u'contributor': u'Microsoft COCO group', u'date_created': u'2015-01-27 09:11:52.357475'}
40504
202654
{u'license': 3, u'file_name': u'COCO_val2014_000000391895.jpg', u'coco_url': u'http://mscoco.org/images/391895', u'height': 360, u'width': 640, u'date_captured': u'2013-11-14 11:18:45', u'flickr_url': u'http://farm9.staticflickr.com/8186/8119368305_4e622c8349_z.jpg', u'id': 391895}
{u'image_id': 203564, u'id': 37, u'caption': u'A bicycle replica with a clock as the front wheel.'}


In [6]:
import json
import os

# combine all images and annotations together
imgs = val['images'] + train['images']
annots = val['annotations'] + train['annotations']

# for efficiency lets group annotations by image
itoa = {}
for a in annots:
    imgid = a['image_id']
    if not imgid in itoa: itoa[imgid] = []
    itoa[imgid].append(a)

# create the json blob
out = []
for i,img in enumerate(imgs):
    imgid = img['id']
    
    # coco specific here, they store train/val images separately
    loc = 'train2014' if 'train' in img['file_name'] else 'val2014'
    
    jimg = {}
    jimg['file_path'] = os.path.join(loc, img['file_name'])
    jimg['id'] = imgid
    
    sents = []
    annotsi = itoa[imgid]
    for a in annotsi:
        sents.append(a['caption'])
    jimg['captions'] = sents
    out.append(jimg)
    
json.dump(out, open('coco_raw.json', 'w'))

In [8]:
# lets see what they look like
print out[-1]

{'captions': [u'The patrons enjoy their beverages at the bar.', u'People having a drink in a basement bar.', u'A group of friends enjoys a drink while sitting at a bar.', u'Group of people drinking wine at a public location.', u'Three women and a man are sitting at a bar'], 'file_path': u'train2014/COCO_train2014_000000475546.jpg', 'id': 475546}
